In [31]:
import pandas as pd

# Load the CSV file
story = pd.read_csv("../data/processed/story/result/story_pt03.csv", sep=',')

# Split the DataFrame into chunks of 1000 rows each
story1 = story.iloc[0:2000]
story2 = story.iloc[2000:]

In [32]:
import openai

OPEN_API_KEY = ''

openai.api_key = OPEN_API_KEY

def get_completion(text, question, answer, level):
    try:
        response = openai.chat.completions.create(
            messages=[
                {"role": "system", "content": "[역할] 당신은 초등학교 국어 선생님입니다. 해당 지문을 확인한 후 4지선다 문제를 출제하세요."},
                {"role": "user", "content": f"[방법]"
                                            f" 아래 주어진 지문과 정해진 질문 내용을 바탕으로 {level} 수준의 4지선다 형태의 객관식 문제 1개를 출제하세요. 질문 항목은 입력값 그대로 출력하면서 반드시 아래 Output의 유형에 맞는 구성으로 만드세요. 요구한 결과물만 출력해줘"
                                            f"[Input] "
                                            f"[지문]  "
                                            f"{text} "
                                            f"[난이도] - {level}"
                                            f"[질문]"
                                            f"{question}"
                                            f"[답]"
                                            f"{answer}"
                                            f""
                                            f"[Output]"
                                            f"{example}"}
            ],
            model="gpt-4o-mini"
        )
        # print(response)
        return response.choices[0].message.content
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

example = """
{
    "question": "",
    "options":  {
        "1":"보기A",
        "2":"보기b",
        "3":"보기c",
        "4":"보기d"
        },
    "answer": 4,
    "reason": "답에 대한 이유"
}
"""

responses = []
level_mapping = {
    1: "미취학아동",
    2: "초등학교 1학년",
    3: "초등학교 2학년",
    4: "초등학교 3학년",
    5: "초등학교 4학년",
    6: "초등학교 5학년",
    7: "초등학교 6학년"
}

for i, row in story2.iterrows():
    if (i+1)%50==0:
        print(i+1)
    education_level = level_mapping.get(row['Difficulty_Level'], "Unknown")
    response = get_completion(row['text'], row['question'],row['answer'], education_level)
    
    if response:
        # print(response)
        responses.append(response)
    else:
        responses.append("Failed to get a response from the API.")
        
# print(responses)

KeyboardInterrupt: 

In [3]:
responses_df = pd.DataFrame(story2[['text', 'question', 'Difficulty_Level']])
responses_df['response'] = responses

responses_df.to_csv("../data/processed/story/dt03_12_2.csv", index=False, encoding='utf-8-sig')

In [5]:
#불러오기
import pandas as pd
import json
from datetime import datetime

# dt02_11_1 = pd.read_csv("../data/processed/story/dt02_11_1.csv", sep=',')
# dt02_11_2 = pd.read_csv("../data/processed/story/dt02_11_2.csv", sep=',')
# dt02_11_3 = pd.read_csv("../data/processed/story/dt02_11_3.csv", sep=',')
# dt02_11_4 = pd.read_csv("../data/processed/story/dt02_11_4.csv", sep=',')
# dt02_11_5 = pd.read_csv("../data/processed/story/dt02_11_5.csv", sep=',')
# dt02_11_6 = pd.read_csv("../data/processed/story/dt02_11_6.csv", sep=',')
dt03_12_1 = pd.read_csv("../data/processed/story/dt03_12_1.csv", sep=',')
dt03_12_2 = pd.read_csv("../data/processed/story/dt03_12_2.csv", sep=',')


# responses_df = pd.concat([dt02_11_1, dt02_11_2, dt02_11_3, dt02_11_4, dt02_11_5, dt02_11_6], ignore_index=True).reset_index(drop=True)
responses_df = pd.concat([dt03_12_1, dt03_12_2], ignore_index=True).reset_index(drop=True)

story = pd.read_csv("../data/processed/story/result/story_all.csv", sep=',')
content_to_textID = dict(zip(story['text'], story['textID']))

print(responses_df['response'].head())
transformed_data = []

def get_textID(content):
    return content_to_textID.get(content, None)

def transform_row(row):
    response_json = json.loads(row['response'])
    options = response_json.get('options', {})
    
    answer_key = str(response_json.get("answer", ""))  
    answer_value = ""
    for k, v in options.items():
        if str(k) == answer_key:
            answer_value = v  
            break
    
    now = datetime.now().isoformat()

    transformed = {
        "title": response_json.get("question", ""),
        "content": row['text'],
        "answer" : answer_value,
        "solution": response_json.get("reason", ""),
        "questionTypeCode": "QT01",
        "problemLevelCode": f"PL0{row['Difficulty_Level']}",
        "problemTypeCode": "PT03",
        "problemTypeDetailCode": "PT0312",
        "metadata": {
            "options": [v for v in options.values()] 
        },
        "createdDate": now,
        "modifiedDate": now
    }
    
    return transformed


for i, row in responses_df.iterrows():
    try:
        json.loads(row['response'])  # Try parsing the original JSON
        transformed_data.append(transform_row(row))
    except json.JSONDecodeError as e:
        print(f"Error in row {i}: {e}")
        print(f"Problematic JSON: {row['response']}")

    
dt03_12_df = pd.DataFrame(transformed_data)

0    {\n    "question": "릴리엔탈과 구스타프는 하늘의 새를 구경하기 위해...
1    {\n    "question": "물방울들은 파란 하늘을 보자 어떤 반응이었나요?...
2    {\n    "question": "페네옹은 점을 찍어 그린 시냐크의 그림을 어떻게...
3    {\n    "question": "프랑스 화가 시냐크는 언제 태어났나요?",\n ...
4    {\n    "question": "쇠라는 어떤 그림 방법을 개발했나요?",\n  ...
Name: response, dtype: object
Error in row 471: Expecting property name enclosed in double quotes: line 11 column 1 (char 284)
Problematic JSON: {
    "question": "토끼가 미련했다면 어떤 일이 벌어졌을까요?",
    "options":  {
        "1":"용왕에게 칭찬을 받았을 것 같아요.",
        "2":"토끼가 간을 뺏겼을 것 같아요.",
        "3":"토끼가 전혀 걱정하지 않았을 것 같아요.",
        "4":"용왕에게 죽임을 당했을 것 같아요."
    },
    "answer": 4,
    "reason": "토끼가 미련했다면 용왕이 화가 나서 토끼를 죽였을 확률이 높기 때문에.",
}
Error in row 965: Expecting property name enclosed in double quotes: line 8 column 5 (char 130)
Problematic JSON: {
    "question": "지구는 어떤 힘이 있을까요?",
    "options":  {
        "1":"바람",
        "2":"중력",
        "3":"전기",
        "4":"물",
    },
    "answer": 2,
    "reason": "지구는 물체를 땅으로 끌어당

In [6]:
print(dt03_12_df)
with open("../data/processed/problem_data/pt0312.json", 'w', encoding='utf-8') as f:
    json.dump(transformed_data, f, indent=4, ensure_ascii=False)
dt03_12_df.to_csv("../data/processed/problem_data/pt0312.csv", index=False, encoding='utf-8-sig')

                                                  title  \
0                릴리엔탈과 구스타프는 하늘의 새를 구경하기 위해 매일 무엇을 했나요?   
1                            물방울들은 파란 하늘을 보자 어떤 반응이었나요?   
2                     페네옹은 점을 찍어 그린 시냐크의 그림을 어떻게 생각하나요?   
3                                 프랑스 화가 시냐크는 언제 태어났나요?   
4                                  쇠라는 어떤 그림 방법을 개발했나요?   
...                                                 ...   
3677                       페르시아 원정길을 나선 병사들은 어떤 모습이었나요?   
3678  함성을 지르며 앞다투어 강물을 건너는 마케도니아 군사들을 보고 페르시아 군은 어떻게...   
3679                            카이사르는 해적들에게 무엇을 들려주었나요?   
3680                      다리를 다 건넌 고슴도치와 다람쥐는 어떻게 했을까요?   
3681             로마 군 진영이 아수라장이 되었을 때, 카르타고 군은 어떻게 됐나요?   

                                                content  \
0     어느 마을에 릴리엔탈과 구스타프라는 형제가 살았어. 릴리엔탈과 구스타프는 어릴 때부...   
1     물방울들은 바다 위로 떠올랐지. "아, 눈부셔! 여기가 어디야?" 파란 하늘에는 해...   
2     그림 속 모델은 시냐크의 친구 펠릭스 페네옹이에요. 페네옹은 점을 찍어 그린 시냐크...   
3     폴 시냐크(1863 1935)는 프랑스의 화가예요. 모네의 그림을 보고 화가가 되기...

In [84]:
import pandas as pd
import json

story = pd.read_csv("../data/processed/story/result/story_all.csv", sep=',')

dt0211 = pd.read_csv("../data/processed/problem_data/pt0211.csv", sep=',')
dt0312 = pd.read_csv("../data/processed/problem_data/pt0312.csv", sep=',')
# dt0211 = dt0211.drop(columns=['isbn', 'textID'])
# dt0312 = dt0312.drop(columns=['isbn', 'textID'])

content_to_textID = dict(zip(story['text'], story['textID']))
content_to_isbn = dict(zip(story['text'], story['isbn']))
content_to_classification = dict(zip(story['text'], story['classification']))
content_to_word_count = dict(zip(story['text'], story['word_count']))
content_to_word_avg = dict(zip(story['text'], story['word_avg']))
content_to_word_hard = dict(zip(story['text'], story['word_hard']))
content_to_length = dict(zip(story['text'], story['length']))
content_to_Difficulty_Level = dict(zip(story['text'], story['Difficulty_Level']))

def get_contentID(content):
    return str(content_to_isbn.get(content))+'_'+str(content_to_textID.get(content))
def get_classification(content):
    return content_to_classification.get(content)
def get_word_count(content):
    return content_to_word_count.get(content)
def get_word_avg(content):
    return content_to_word_avg.get(content)
def get_word_hard(content):
    return content_to_word_hard.get(content)
def get_length(content):
    return content_to_length.get(content)
def get_Difficulty_Level(content):
    return content_to_Difficulty_Level.get(content)

dt0211['contentID'] = dt0211['content'].apply(get_contentID)
dt0211['classification'] = dt0211['content'].apply(get_classification)
dt0211['word_count'] = dt0211['content'].apply(get_word_count)
dt0211['word_avg'] = dt0211['content'].apply(get_word_avg)
dt0211['word_hard'] = dt0211['content'].apply(get_word_hard)
dt0211['length'] = dt0211['content'].apply(get_length)
dt0211['Difficulty_Level'] = dt0211['content'].apply(get_Difficulty_Level)

dt0312['contentID'] = dt0312['content'].apply(get_contentID)
dt0312['classification'] = dt0312['content'].apply(get_classification)
dt0312['word_count'] = dt0312['content'].apply(get_word_count)
dt0312['word_avg'] = dt0312['content'].apply(get_word_avg)
dt0312['word_hard'] = dt0312['content'].apply(get_word_hard)
dt0312['length'] = dt0312['content'].apply(get_length)
dt0312['Difficulty_Level'] = dt0312['content'].apply(get_Difficulty_Level)

print(dt0211)
print(dt0312)


                                            title  \
0                                릴리엔탈은 누구와 형제인가요?   
1                       그림 속에서 빠르게 걷고 있는 건 누구인가요?   
2                   클림트의 작품에서 남녀 한 쌍은 무엇을 하고 있나요?   
3                     클림트의 키스라는 작품이 전시된 곳은 어디인가요?   
4                            클림트의 아버지 직업은 무엇이었나요?   
...                                           ...   
5547     카이사르 군대를 당해 낼 수 없었던 게르만인들은 어디를 건너 도망쳤나요?   
5548                       위인전을 읽었을 때 어떤 느낌이 들까요?   
5549    많은 친구들은 위인전과 같은 책을 보면서 마음속으로 무엇을 정하게 될까요?   
5550            워브는 굵은 생나무를 성냥개비처럼 쓰러뜨린 곳은 어디인가요?   
5551  고개를 들지 않는 데모스테네스를 기다리다 지쳐 어깨를 흔든 사람은 누구일까요?   

                                                content            answer  \
0     어느 마을에 릴리엔탈과 구스타프라는 형제가 살았어. 릴리엔탈과 구스타프는 어릴 때부...              구스타프   
1     그림 속에서 빠르게 걷고 있는 보이지요. 걷고 있는 개의 여러 여러 동작을 한 그림...                 개   
2     키스. 구스타프 클림트, 1907 1908년, 벨베델 오스트리아 미술관. 황금 옷을...  꽃밭 위에서 키스하고 있어요.   
3     키스. 구스타프 클림트, 1907 1908년, 벨베델 오스트리아 미술관. 황금 옷을...

In [85]:
from datetime import datetime
def transform_row(row):
    now = datetime.now().isoformat()

    transformed = {
        "title": row['title'],
        "content": row['content'],
        "answer" : row['answer'],
        "solution": row['solution'],
        "questionTypeCode": row['questionTypeCode'],
        "problemLevelCode": row['problemLevelCode'],
        "problemTypeCode": row['problemTypeCode'],
        "problemTypeDetailCode": row['problemTypeDetailCode'],
        "metadata": row['metadata'],
        "createdDate": now,
        "modifiedDate": now,
        "contentID": row['contentID'],
        "classification": row['classification'],
        "wordCount" : row['word_count'],
        "wordAvg" : row['word_avg'],
        "wordHard" : row['word_hard'],
        "length" : row['length'],
        "Difficulty_Level" : row['Difficulty_Level']
    }
    
    return transformed

In [86]:
transformed_data = []
for i, row in dt0211.iterrows():
    try:
        transformed_data.append(transform_row(row))
    except json.JSONDecodeError as e:
        print(f"Error in row {i}: {e}")
dt0211_df = pd.DataFrame(transformed_data)

transformed_data2 = []
for i, row in dt0312.iterrows():
    try:
        transformed_data2.append(transform_row(row))
    except json.JSONDecodeError as e:
        print(f"Error in row {i}: {e}")
dt0312_df = pd.DataFrame(transformed_data2)

In [87]:
with open("../data/processed/problem_data/pt0211.json", 'w', encoding='utf-8') as f:
    json.dump(transformed_data, f, indent=4, ensure_ascii=False)
dt0211.to_csv("../data/processed/problem_data/pt0211.csv", index=False, encoding='utf-8-sig')
# 
with open("../data/processed/problem_data/pt0312.json", 'w', encoding='utf-8') as f:
    json.dump(transformed_data2, f, indent=4, ensure_ascii=False)
dt0312.to_csv("../data/processed/problem_data/pt0312.csv", index=False, encoding='utf-8-sig')